In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *
from setting import *

In [ ]:
w = pd.read_table(PATH_DICT["w.tsv"], index_col=0)

h = pd.read_table(PATH_DICT["h.tsv"], index_col=0)

In [ ]:
if ELEMENT_ENTROPY_QUANTILE < 1:

    feature_entropy = w.apply(ccal.compute_entropy, axis=1)

    w = w.loc[
        feature_entropy.index[
            feature_entropy < feature_entropy.quantile(q=ELEMENT_ENTROPY_QUANTILE)
        ]
    ]

    print(w.shape)

    sample_entropy = h.apply(ccal.compute_entropy)

    h = h[
        sample_entropy.index[
            sample_entropy < sample_entropy.quantile(q=ELEMENT_ENTROPY_QUANTILE)
        ]
    ]

    print(h.shape)

In [ ]:
gps_map = ccal.GPSMap(
    w=w.T,
    h=h,
    w_pull_power=GPS_MAP_W_PULL_POWER,
    h_pull_power=GPS_MAP_H_PULL_POWER,
    plot=False,
)

In [ ]:
for element_type in ("feature", "sample"):

    if element_type == "feature":

        element_alias = FEATURE_ALIAS

        w_or_h = "w"

        element_marker_size = GPS_MAP_W_ELEMENT_MARKER_SIZE

    elif element_type == "sample":

        element_alias = SAMPLE_ALIAS

        w_or_h = "h"

        element_marker_size = GPS_MAP_H_ELEMENT_MARKER_SIZE

    title = "{}<br>{}".format(FEATURE_X_SAMPLE_ALIAS, element_alias)

    gps_map.plot_gps_map(
        w_or_h,
        title=title,
        element_marker_size=element_marker_size,
        html_file_path="{}/map.html".format(PATH_DICT["{}/".format(w_or_h)]),
    )

    hcc__k_x_column_file_path = PATH_DICT["{}|hcc__k_x_column.tsv".format(w_or_h)]

    if os.path.isfile(hcc__k_x_column_file_path):

        hcc__k_x_column = pd.read_table(hcc__k_x_column_file_path, index_col=0)

        if element_type == "feature":

            hcc_k = W_HCC_K

            elements = gps_map.w_elements

            bandwidth_factor = GPS_MAP_W_BANDWIDTH_FACTOR

        elif element_type == "sample":

            hcc_k = H_HCC_K

            elements = gps_map.h_elements

            bandwidth_factor = GPS_MAP_H_BANDWIDTH_FACTOR

        gps_map.set_element_states(
            w_or_h,
            hcc__k_x_column.loc["K{}".format(hcc_k), elements],
            bandwidth_factor=bandwidth_factor,
        )

        gps_map.plot_gps_map(
            w_or_h,
            title=title,
            element_marker_size=element_marker_size,
            html_file_path="{}/map.html".format(
                PATH_DICT["{}|hcc|map/".format(w_or_h)]
            ),
            plotly_file_path="{}/{} Map.html".format(
                PATH_DICT["plotly/"], element_alias
            ),
        )

In [ ]:
ccal.dump_gps_map(gps_map, PATH_DICT["gps_map.pickle.gz"])